A neural network consist of cnn layer (Yoon Kim,2014) and 4 fully connected layers.

Source: https://github.com/jojonki/cnn-for-sentence-classification





In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os
os.chdir('/content/drive/MyDrive/sharif/DeepLearning/ipython(guide)')

In [3]:
import numpy as np
import codecs
import os
import random
import pandas
from keras import backend as K
from keras.models import Model
from keras.layers.embeddings import Embedding
from keras.layers import Input, Dense, Lambda, Permute, Dropout,Softmax
from keras.layers import LSTM, MaxPooling1D
from keras.optimizers import SGD
import ast
import re
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import train_test_split
import gensim
from keras.models import load_model
from keras.callbacks import EarlyStopping, ModelCheckpoint

In [4]:
limit_number = 750
data = pandas.read_csv('../Data/limited_to_'+str(limit_number)+'.csv',index_col=0,converters={'body': eval})
data = data.dropna().reset_index(drop=True)
X = data["body"].values.tolist()
y = pandas.read_csv('../Data/limited_to_'+str(limit_number)+'.csv')
labels = []
tag=[]
for item in y['tag']:
  labels += [i for i in re.sub('\"|\[|\]|\'| |=','',item.lower()).split(",") if i!='' and i!=' ']
  tag.append([i for i in re.sub('\"|\[|\]|\'| |=','',item.lower()).split(",") if i!='' and i!=' '])
labels = list(set(labels))
mlb = MultiLabelBinarizer()
Y=mlb.fit_transform(tag)

In [5]:
len(labels)

78

In [6]:
sentence_maxlen = max(map(len, (d for d in X)))
print('sentence maxlen', sentence_maxlen)

sentence maxlen 300


In [7]:
freq_dist = pandas.read_csv('../Data/FreqDist_sorted.csv',index_col=False)
vocab=[]
for item in freq_dist["word"]:
  try:
    word=re.sub(r"[\u200c-\u200f]","",item.replace(" ",""))
    vocab.append(word)
  except:
    pass
  
print(vocab[10])

زبان


In [8]:
vocab = sorted(vocab)
vocab_size = len(vocab)

In [9]:
print('vocab size', len(vocab))
w2i = {w:i for i,w in enumerate(vocab)}
# i2w = {i:w for i,w in enumerate(vocab)}
print(w2i["زبان"])

vocab size 225345
129280


In [10]:
def vectorize(data, sentence_maxlen, w2i):
    vec_data = []
    
    for d in data:
       
        vec = [w2i[w] for w in d if w in w2i]
        pad_len = max(0, sentence_maxlen - len(vec))
        vec += [0] * pad_len
        vec_data.append(vec)
        # print(d)
        
    vec_data = np.array(vec_data)
    
    return vec_data

vecX = vectorize(X, sentence_maxlen, w2i)
vecY=Y

In [11]:
X_train, X_test, y_train, y_test = train_test_split(vecX, vecY, test_size=0.2)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25)
print('train: ', X_train.shape , '\ntest: ', X_test.shape , '\nval: ', X_val.shape ,"\ny_tain:",y_train.shape )
# print(vecX[0])

train:  (12935, 300) 
test:  (4312, 300) 
val:  (4312, 300) 
y_tain: (12935, 78)


In [12]:
embd_dim = 300


# ***If the word2vec model is not generated before, we should run the next block.***

In [13]:
# embed_model = gensim.models.Word2Vec(X, size=embd_dim, window=5, min_count=5)
# embed_model.save('word2vec_model')

# ***Otherwise, we can run the next block.***

In [14]:
embed_model=gensim.models.Word2Vec.load('word2vec_model')

In [15]:
word2vec_embd_w = np.zeros((vocab_size, embd_dim))
for word, i in w2i.items():
  if word in embed_model.wv.vocab:
        embedding_vector =embed_model[word]
  
        # words not found in embedding index will be all-zeros.
        word2vec_embd_w[i] = embedding_vector

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  after removing the cwd from sys.path.


In [ ]:
def Net(vocab_size, embd_size, sentence_maxlen, glove_embd_w):
    sentence = Input((sentence_maxlen,), name='SentenceInput')
    
    # embedding
    embd_layer = Embedding(input_dim=vocab_size, 
                           output_dim=embd_size, 
                           weights=[word2vec_embd_w], 
                           trainable=False,
                           name='shared_embd')
    embd_sentence = embd_layer(sentence)
    # embd_sentence = Permute((2,1))(embd_sentence)
    # embd_sentence = Lambda(lambda x: K.expand_dims(x, -1))(embd_sentence)
    
    # lstm
    lstm_1 = LSTM(128, return_sequences=True,name='lstm_1')(embd_sentence)
    # drop_out_1 = Dropout(0.5,name='drop_out_1')(lstm_1)
    lstm_2 = LSTM(64,name='lstm_2')(lstm_1)
    drop_out_2 = Dropout(0.4,name='drop_out_2')(lstm_2)

    # dense
    dense_1 = Dense(256,name='dense_1')(drop_out_2)
    dense_2 = Dense(128,name='dense_2')(dense_1)
    dense_3 = Dense(len(labels),activation='sigmoid',name='dense_3')(dense_1)
    out = Softmax(name='out')(dense_3) 
    
    sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
    model = Model(inputs=sentence, outputs=out, name='sentence_claccification')
    model.compile(optimizer=sgd, loss='binary_crossentropy', metrics=["accuracy",
    "categorical_accuracy",]) 
    return model


model = Net(vocab_size, embd_dim, sentence_maxlen,word2vec_embd_w)
print(model.summary())

Model: "sentence_claccification"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
SentenceInput (InputLayer)   [(None, 300)]             0         
_________________________________________________________________
shared_embd (Embedding)      (None, 300, 300)          67603500  
_________________________________________________________________
lstm_1 (LSTM)                (None, 300, 128)          219648    
_________________________________________________________________
lstm_2 (LSTM)                (None, 64)                49408     
_________________________________________________________________
drop_out_2 (Dropout)         (None, 64)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 256)               16640     
_________________________________________________________________
dense_3 (Dense)              (None, 128)   

In [ ]:
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=10) # Model stop training after 50 epoch where validation loss didnt decrease
mc = ModelCheckpoint('best_cnn_4fc.h5', monitor='val_loss', mode='min', verbose=1, save_best_only=True) #You save model weight at the epoch where validation loss is minimal
model.fit(X_train, y_train, batch_size=32,epochs=50,verbose=1,validation_data=(X_val, y_val),callbacks=[es,mc])#you can run for 1000 epoch btw model will stop after 50 epoch without better validation loss

Epoch 1/50
405/405 [==============================] - 263s 643ms/step - loss: 0.4224 - accuracy: 0.0095 - categorical_accuracy: 0.0095 - val_loss: 0.1085 - val_accuracy: 0.0227 - val_categorical_accuracy: 0.0227

Epoch 00001: val_loss improved from inf to 0.10851, saving model to best_cnn_4fc.h5
Epoch 2/50
405/405 [==============================] - 270s 666ms/step - loss: 0.1098 - accuracy: 0.0182 - categorical_accuracy: 0.0182 - val_loss: 0.1081 - val_accuracy: 0.0028 - val_categorical_accuracy: 0.0028

Epoch 00002: val_loss improved from 0.10851 to 0.10813, saving model to best_cnn_4fc.h5
Epoch 3/50
405/405 [==============================] - 271s 670ms/step - loss: 0.1096 - accuracy: 0.0178 - categorical_accuracy: 0.0178 - val_loss: 0.1081 - val_accuracy: 0.0174 - val_categorical_accuracy: 0.0174

Epoch 00003: val_loss improved from 0.10813 to 0.10809, saving model to best_cnn_4fc.h5
Epoch 4/50
405/405 [==============================] - 269s 664ms/step - loss: 0.1110 - accuracy: 0.01

# ***If the model is generated before:***

In [16]:
model = load_model('Lstm_no_binary.h5')
# model.save('Lstm_no_binary.h5')

In [17]:
pred=model.predict(X_test)
# For evaluation: If the probability > 0.5 you can say that it belong to the class.

In [18]:
print(pred[0])#example

[0.02795699 0.03632668 0.03054592 0.00451815 0.01891431 0.00402513
 0.03460523 0.009386   0.0056994  0.02048925 0.03409892 0.01699072
 0.02841347 0.00396761 0.00411966 0.03153625 0.02900848 0.03963751
 0.02502877 0.01624849 0.02501237 0.03353897 0.00889233 0.03062406
 0.01693025 0.03100514 0.0305661  0.03497058 0.02809665 0.03085032
 0.01194304 0.0202837  0.03029352 0.03475657 0.01902238 0.0304139
 0.03192711 0.01743361 0.01185411 0.00687701 0.03619859 0.02682042
 0.02025905 0.03725979 0.03020337 0.00473198 0.0087533  0.03587213
 0.033048   0.01698637 0.00688046 0.0048095  0.00941154 0.02183151
 0.03145954 0.00770155 0.01462647 0.01370907 0.04060733 0.03438759
 0.03726155 0.01066977 0.03439572 0.03662232 0.01975405 0.03599265
 0.00838968 0.02445996 0.03207257 0.03444019 0.02909622 0.03571284
 0.02798748 0.0099999  0.00622311 0.03072423 0.01653567 0.02828187]


In [21]:
y_pred=[]
for l in pred:
  temp=[]
  for value in l:
    if value>= np.mean(pred[0]) + .75*np.sqrt(np.var(pred[0])):
      temp.append(1)
    else:
      temp.append(0)
  y_pred.append(temp)

In [22]:
from sklearn.metrics import classification_report,accuracy_score

print("accuracy=",accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))

accuracy= 0.0
              precision    recall  f1-score   support

           0       0.00      0.00      0.00       117
           1       0.04      1.00      0.07       155
           2       0.00      0.00      0.00       148
           3       0.00      0.00      0.00        18
           4       0.00      0.00      0.00        76
           5       0.00      0.00      0.00        18
           6       0.04      1.00      0.07       151
           7       0.00      0.00      0.00        52
           8       0.00      0.00      0.00        25
           9       0.00      0.00      0.00        98
          10       0.04      1.00      0.07       159
          11       0.00      0.00      0.00        83
          12       0.00      0.00      0.00       143
          13       0.00      0.00      0.00        31
          14       0.00      0.00      0.00        17
          15       0.04      1.00      0.07       153
          16       0.00      0.00      0.00       115
          17 

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
